In [ ]:
!unzip '/content/archive.zip'

Archive:  /content/archive.zip
replace Entity Recognition in Resumes.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import json

In [ ]:
pip install spacy

In [ ]:
import requests

In [ ]:
with open('/content/traindata.json','r') as f:
  lines = f.readlines()

In [ ]:
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

In [ ]:
training_data = convert_dataturks_to_spacy('/content/Entity Recognition in Resumes.json')

In [ ]:
training_data[0][1]

{'entities': [(1296, 1622, 'Skills'),
  (993, 1154, 'Skills'),
  (939, 957, 'College Name'),
  (883, 905, 'College Name'),
  (856, 860, 'Graduation Year'),
  (771, 814, 'College Name'),
  (727, 769, 'Designation'),
  (407, 416, 'Companies worked at'),
  (372, 405, 'Designation'),
  (95, 145, 'Email Address'),
  (60, 69, 'Location'),
  (49, 58, 'Companies worked at'),
  (13, 46, 'Designation'),
  (0, 12, 'Name')]}

In [ ]:
import spacy
import random

In [ ]:
nlp = spacy.blank('en')

def train_model(train_data):
  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner,last = True)

    for text,annotation in training_data:
      for ent in annotation['entities']:
        ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
      optimizer = nlp.begin_training()
      for itn in range(10):
        print('Starting iterations'+str(itn))
        random.shuffle(training_data)
        loss = {}
        index=0
        for text,annotation in training_data:
          try:
            nlp.update(
                [text],
                [annotation],
                drop=0.2,
                sgd = optimizer,
                losses=loss)
          except Exception as e:
            pass
        print(loss)
  
            

In [ ]:
train_model(training_data)

Starting iterations0
{'ner': 20140.110715183117}
Starting iterations1
{'ner': 16555.960930897283}
Starting iterations2
{'ner': 12217.728864604438}
Starting iterations3
{'ner': 9467.625958431092}
Starting iterations4
{'ner': 7719.760303009499}
Starting iterations5
{'ner': 7649.206090428478}
Starting iterations6
{'ner': 7230.383627521222}
Starting iterations7
{'ner': 6122.915956017891}
Starting iterations8
{'ner': 6173.087240941312}
Starting iterations9
{'ner': 5858.423052201798}


In [ ]:
nlp.to_disk('nlp_model')

In [ ]:
nlp_model = spacy.load('nlp_model')

In [ ]:
doc = nlp_model(training_data[0][0])
for ent in doc.ents:
  print(ent.label_ + ' - ' + ent.text)

Name - Vishwanath P
Designation - Senior Executive
Companies worked at - MIS & Audit) - Job Profile in LabourNet Services India Pvt Ltd
Location - Bengaluru
Email Address - indeed.com/r/Vishwanath-P/06a16ac2d087d3c9
Designation - Senior Executive
Companies worked at - ANI Technologies
Graduation Year - 2016
Designation - HR Executive
Companies worked at - Accenture
Companies worked at - INFOSYS
Companies worked at - Associate
Companies worked at - Infosys
Companies worked at - Infosys
Degree - BSc
College Name - V V Pura College of Science


In [ ]:
pip install pymupdf

     |████████████████████████████████| 8.7 MB 7.6 MB/s 


In [ ]:
def extractor(resume):
  import fitz
  res = fitz.open(resume)
  text=""
  for page_num in range(res.page_count):
    obj = res[page_num]
    text += obj.getText('text')
    return text

In [ ]:
resume = extractor('/content/Karan Katariya_PI032_NITJSR.pdf')

Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [ ]:
def cleanResume(resumeText):
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('\n', '  ', resumeText)  # remove mentions
    return resumeText

In [ ]:
resume = cleanResume(resume)

In [ ]:
doc = nlp_model(resume)
for ent in doc.ents:
  print(ent.label_ + ' - ' + ent.text)

Name - Karan Katariya
Degree - B.tech in Production and Industrial
College Name - National Institute of Technology, Jamshedpur
Graduation Year - 2018
Designation - Data Science And Analyst Intern
Companies worked at - An
Skills - Python   Data Analysis   Data Visualization


In [ ]:
test = extractor('/content/candidate_004.pdf')

Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [ ]:
doc1 = nlp_model(test)
for ent in doc1.ents:
  print(ent.label_ + ' - ' + ent.text)

Name - OLIVIA SANTOS
Location - Cognizant
Companies worked at - 2019
Software Developer Analyst
Worked as a software developer and analyst for various
projects concerned with middleware. Actively participated
in the development cycle and end to end needs of the
projects.
ACADEMIC PROFILE
B.tech Electrical Engineering, 2018
Bhubaneshwar Institute of Technology
EXECUTIVE
SUMMARY
Data Engineer and solution
specialist. Focused on using data
building and data driven
production ready procedures.
PERSONAL
SKILLS
Machine Learning, Data Analytics,
Project Management, Software
Development, Agile Methodology,
Business Growth.
EXTRA-CURRICULARS
EXTRA-CURRICULARS

